In [ ]:
import gdsfactory as gf
from blocks import *
from comb_drive_tuning import *
from dowhen import do, when

In [ ]:
p={'WG':3,'PADDING':2,'DEEP_ETCH':1,'DEEP_ETCH_PL':1}

In [ ]:
xs_metal_wire = metal_wire(core_width=10.32, wire_width=8.32, mask_offset=5, deep_etch_layer="DEEP_ETCH_PL")
xs_metal_wire_wide = metal_wire(core_width=60, wire_width=50, mask_offset=5 , deep_etch_layer="DEEP_ETCH_PL")
xs = cross_section_with_sleeves(0.43,5,radius=5,radius_min=5)
xs_routing = cross_section_with_sleeves(0.43,5,radius=20, radius_min=15)
Xtrans = gf.path.transition(cross_section1=xs_metal_wire, cross_section2=xs_metal_wire_wide, width_type="linear",offset_type="linear",)
gc_spec = partial(gf.components.grating_coupler_elliptical,wavelength=1.55,nclad=1,neff=2.3,n_periods=60,fiber_angle=8)
my_coupler_spec = partial(my_coupler,coupler=gc_spec,waveguide_width=0.43)

In [ ]:
def doubly_clamped_beam_with_ring_resonator(beam_length=500, beam_width=0.1,spring_length=30, spring_width=0.16,finger_num = 121, waveguide_spacing=100):
    x_span_resonator = 10
    beam_spec = partial(doubly_clamped_beam_with_round_support,length=beam_length, width=beam_width,support_length=0.2,create_mask=True,mask_offset=6)
    spring_spec = partial(spring_pair_anchor_outside,spring_length=spring_length,spring_width=spring_width,mask_offset=10,spring_separation=4)
    spring_beam = doubly_clamped_beam_with_spring(beam_spec,spring_spec)
    spring_beam = tmp_merge_deep_etch_mask(spring_beam)
    # create_deep_etch_mask(spring_beam,mask_offset=10,y_off=True,x_off=False,deep_etch_layer='PROTECTION_PL')
    c = gf.Component()
    # bridge_ref = c << bridge(mxn=(5,5),mask_offset=5)
    spring_beam_ref = c << spring_beam
    # bridge_ref.connect("E1", spring_beam_ref.ports["w1"],allow_width_mismatch=True)
    mfs_length = max(finger_num*(0.16+0.3),  spring_spec().info['frame_length'])
    mfs_ref = c << movable_finger_support(length=mfs_length,mask_offset=10,open=['left','right'])
    mfs_ref.connect("E1", spring_beam_ref.ports["w1"],allow_width_mismatch=True)
    comb = combdrive_fingers(fingers=finger_num,finger_length=3.2,finger_gap=0.3,thickness=0.16,base_thickness=0.08,a_c=3,mask_offset=10,base_length=mfs_length-0.3)
    comb_ref = c << comb
    comb_ref.connect("e1", mfs_ref.ports["W1"],allow_width_mismatch=True)
    fhs = finger_hard_support(size=(10,mfs_length+0.16),mask_offset=10, metal_offset=3, metal_layer='MTOP')
    fhs_ref = c << fhs
    fhs_ref.connect("E1", comb_ref.ports["w1"],allow_width_mismatch=True, allow_type_mismatch=True)
    # fhs_ref2 = c << fhs
    # fhs_ref2.connect("E1", spring_beam_ref.ports["e1"],allow_width_mismatch=True)
    pad_ = pad(size=(400,400), metal_offset=10,pad_layer='PADDING')
    pad_ref = c << pad_
    pad_ref.connect("E1", fhs_ref.ports["W1"],allow_width_mismatch=True, allow_type_mismatch=True)
    pad_ref.move((-100,500))
    routing_with_mytaper(c, pad_ref.ports['S1'].copy(trans=gf.kdb.Trans(x=150*1000,y=0)), fhs_ref.ports['W1'], xs_metal_wire_wide, fhs.cross_section, taper_length=20)
    
    
    
    with do("create_deep_etch_mask(c,'bbox',mask_offset=mask_offset, deep_etch_layer='DEEP_ETCH_PL')").when(beam_fixed_support, "create_deep_etch_mask").goto("for port in rect.ports"):
        bfs_ref = c << beam_fixed_support(size=(200,200), metal_offset=5, mask_offset=10)
    bfs_ref.connect("E1", spring_beam_ref.ports["e1"],allow_width_mismatch=True)
    
    ring_beam_spacer = vertical_spacer(length=0.1)
    ring_ref = c << ring_resonator_fill_middle(gap=0.3, radius=10, length_x=3,length_y=0,cross_section=xs,bend="bend_circular", x_span=x_span_resonator)
    spacer_ref = c << ring_beam_spacer
    spacer_ref.connect('p1', spring_beam_ref.ports['s1'],allow_width_mismatch=True)
    ring_ref.connect('p1', spacer_ref.ports['p2'],allow_width_mismatch=True)
    ring_ref.movex(beam_length/4)
    o1_ring = ring_ref['o1']
    o2_ring = ring_ref['o2']
    c.add_port(
        name='optical_in',
        width=o1_ring.width,
        orientation=-90,
        center=(o1_ring.center[0] - (waveguide_spacing- x_span_resonator)/2, o1_ring.center[1] -50),
        layer=o1_ring.layer,
        port_type='optical'
    )
    c.add_port(
        name='optical_out',
        width=o2_ring.width,
        orientation=-90,
        center=(o2_ring.center[0] + (waveguide_spacing- x_span_resonator)/2, o2_ring.center[1] -50),
        layer=o2_ring.layer,
        port_type='optical'
    )
    copied_portin = c.ports.filter(regex='optical_in')[0].copy()
    copied_portout = c.ports.filter(regex='optical_out')[0].copy()
    copied_portin.name = 'optical_in_reverse'
    copied_portout.name = 'optical_out_reverse'
    copied_portin.orientation = 90
    copied_portout.orientation = 90
    r1 = gf.routing.route_single(
        c,
        port1=o1_ring,
        port2=copied_portin,
        cross_section=xs_routing,
        )
    r2 = gf.routing.route_single(
        c,
        port1=o2_ring,
        port2=copied_portout,
        cross_section=xs_routing,
    )
    
    spring_pad = pad(size=(400, 400), metal_offset=10,pad_layer='PADDING')
    spring_pad_ref = c << spring_pad
    spring_pad_ref.movex(origin=spring_pad_ref.ports['S1'].x, destination=spring_beam_ref.ports['up_anchor'].x)
    spring_pad_ref.movey(origin=spring_pad_ref.ymax, destination=pad_ref.ymax)
    routing_with_mytaper(
        c,
        port1 = spring_pad_ref.ports["S1"],
        port2 = spring_beam_ref.ports['up_anchor'],
        cross_section1=xs_metal_wire_wide,
        cross_section2=xs_metal_wire,
    )
    spring_pad_ref.movex(origin=spring_pad_ref.xmin, destination=spring_beam_ref.ports['up_anchor'].x-50)
  
    
    # put all parameter values to the component info
    c.info['beam_length'] = beam_length
    c.info['beam_width'] = beam_width
    c.info['spring_length'] = spring_length
    c.info['spring_width'] = spring_width
    c.info['finger_num'] = finger_num
    
    c =merge_layers_with_priority(c, p)
    c = EBL_PL_overlap(c)
    
    c.info['params'] = {
        'beam_length': beam_length,
        'beam_width': beam_width,
        'spring_length': spring_length,
        'spring_width': spring_width,
        'finger_num': finger_num
    }
    
    return c 
c = doubly_clamped_beam_with_ring_resonator(finger_num=101, spring_length=15,beam_length=500)
c.show()

In [ ]:
def beam_with_ring_resonator_3_combs(beam_length=400, beam_width=0.1,spring_length=50, spring_width=0.16,finger_num = 70, electrode_gap = 2,finger_num_sub=30, waveguide_spacing=100):
    x_span_resonator = 10
    beam_spec = partial(doubly_clamped_beam_with_round_support,length=beam_length, width=beam_width,support_length=0.2,create_mask=True,mask_offset=6)
    spring_spec = partial(spring_pair_anchor_outside,spring_length=spring_length,spring_width=spring_width,mask_offset=10,spring_separation=4, open=['left','right'])
    
    
    c = gf.Component()
    
    beam_ref = c << beam_spec()
    
    beam_ref.movey(origin=(beam_ref.ymax+beam_ref.ymin)/2,destination=0)
    # create_deep_etch_mask(c, deep_etch_layer='PROTECTION_PL',x_off=False,mask_offset=10)
    bridge1_ref = c << bridge(mxn=(4,3),open=['left','top','bottom'])
    bridge1_ref.connect("E1", beam_ref.ports["w1"],allow_width_mismatch=True)
    bridge2_ref = c << bridge(mxn=(4,20),open=['left','right'])
    bridge2_ref.connect("E1", bridge1_ref.ports["W1"],allow_width_mismatch=True)
    finger_sub = combdrive_fingers(fingers=finger_num_sub,finger_length=3.2,finger_gap=0.3,thickness=0.2,base_thickness=0,a_c=3,mask_offset=10)
    comb1_ref = c << finger_sub
    mfs_1_ref = c << movable_finger_support(length=ceil(finger_sub.info["base_length"])+5,mask_offset=10,open=["bottom"])
    
    mfs_1_ref.connect("S1", bridge1_ref.ports["N1"],allow_width_mismatch=True)
    comb1_ref.connect("e1", mfs_1_ref.ports["W1"],allow_width_mismatch=True)
    comb1_ref.movey(2.22)
    # fhs_sub1 = finger_hard_support_L(size=(10,finger_num_sub*(0.2+0.3)+10),mask_offset=10, metal_offset=1, metal_layer='MTOP')
    fhs_sub1 = finger_hard_support_L(size=(10.32,spring_spec().info['frame_length']/2),mask_offset=10, metal_offset=1, metal_layer='MTOP')
    fhs_sub1_ref = c << fhs_sub1
    fhs_sub1_ref.connect("E1", comb1_ref.ports["w1"],allow_width_mismatch=True, allow_type_mismatch=True)
    fhs_sub1_ref.movey(origin=fhs_sub1_ref.ymin+10, destination=comb1_ref.ports["w1"].y-finger_sub.info["base_length"]/2)
    comb2_ref = c << finger_sub
    mfs_2_ref = c << movable_finger_support(length=ceil(finger_sub.info["base_length"])+5,mask_offset=10,open=["top"])
    mfs_2_ref.connect("N1", bridge1_ref.ports["S1"],allow_width_mismatch=True)
    # mfs_2_ref.movex(5)
    comb2_ref.connect("e1", mfs_2_ref.ports["W1"],allow_width_mismatch=True)
    comb2_ref.movey(-1.8)
    fhs_sub2 = finger_hard_support_L(size=(10.32,spring_spec().info['frame_length']/2),mask_offset=10, metal_offset=1, metal_layer='MTOP')
    fhs_sub2_ref = c << fhs_sub2
    fhs_sub2_ref.connect("W1", comb2_ref.ports["w1"],allow_width_mismatch=True, allow_type_mismatch=True)
    fhs_sub2_ref.movey(origin=fhs_sub2_ref.ymax-10, destination=comb2_ref.ports["w1"].y+finger_sub.info["base_length"]/2)

    bridge3_ref = c << bridge(mxn=(4,1),open=['right','top','bottom','left'])
    bridge3_ref.connect("E1", bridge2_ref.ports["W1"],allow_width_mismatch=True)
    bridge3_up_ref = c << bridge(mxn=((spring_spec().info['frame_length']-4)/2,1),open=['bottom','left'])
    bridge3_up_ref.connect("S1", bridge3_ref.ports["N1"],allow_width_mismatch=True)
    spring_ref = c << spring_spec()
    spring_ref.connect("e1", bridge3_ref.ports["W1"],allow_width_mismatch=True)
    bridge3_down_ref = c << bridge(mxn=((spring_spec().info['frame_length']-4)/2,1),open=['top','left'])
    bridge3_down_ref.connect("N1", bridge3_ref.ports["S1"],allow_width_mismatch=True)
    
    mfs_length = max(finger_num*(0.16+0.3),  spring_spec().info['frame_length'])
    mfs_ref = c << movable_finger_support(length=mfs_length,mask_offset=10,open=['right'])
    mfs_ref.connect("E1", spring_ref.ports["w1"],allow_width_mismatch=True)
    comb = combdrive_fingers(fingers=finger_num,finger_length=3.2,finger_gap=0.3,thickness=0.16,base_thickness=0.01,a_c=3,mask_offset=10,base_length=mfs_length-0.3)
    comb_ref = c << comb
    comb_ref.connect("e1", mfs_ref.ports["W1"],allow_width_mismatch=True)
    comb_ref.movey(-0.25)
    
    
    fhs = finger_hard_support(size=(10,mfs_length+0.16),mask_offset=18, metal_offset=1, metal_layer='MTOP')
    fhs_ref = c << fhs
    fhs_ref.connect("E1", comb_ref.ports["w1"],allow_width_mismatch=True, allow_type_mismatch=True)
    
    pad_ = U_shape_pad(p1_size=(200,600), p2_size=(500,400), p3_size=(150,500), metal_offset=10)
    pad_ref = c << pad_
    pad_ref.connect("e9", fhs_ref.ports["W1"],allow_width_mismatch=True, allow_type_mismatch=True)
    pad_ref.movex(origin=pad_ref.ports['e11'].x, destination=spring_ref.ports['up_anchor'].x)
    pad_ref.movey(origin=pad_ref.ports['e11'].y, destination=spring_ref.ports['up_anchor'].y+650)
    routing_with_mytaper(c, pad_ref.ports['e12'], fhs_ref.ports['W1'], xs_metal_wire_wide, fhs.cross_section, taper_length=20)
    
    # gf.routing.route_single(c, fhs_ref.ports['W1'],pad_ref.ports['e9'], cross_section=xs_metal_wire_wide, allow_width_mismatch=True, auto_taper=False,port_type='electrical')
    
    
    
    gf.routing.route_single(
        c, 
        fhs_sub1_ref.ports['N1'].copy(trans=gf.kdb.Trans(x =50*1000 ,y=100*1000),post_trans=gf.kdb.Trans.R90), 
        fhs_sub1_ref.ports['N1'], 
        cross_section=xs_metal_wire, 
        allow_width_mismatch=True, 
        auto_taper=False,
    )
    routing_with_mytaper(
        c,
        pad_ref.ports['e10'],
        fhs_sub1_ref.ports['N1'].copy(trans=gf.kdb.Trans(x =50*1000 ,y=100*1000),post_trans=gf.kdb.Trans.R270),
        cross_section1 = xs_metal_wire_wide,
        cross_section2 = xs_metal_wire,
    )    
    
    pad_e10_moved = pad_ref.ports['e10'].copy()
    pad_e10_moved.center = (fhs_sub1_ref.ports['N1'].center[0], pad_ref.ports['e10'].center[1])
        
    c.add_ports([
        fhs_sub1_ref.ports['N1'].copy(trans=gf.kdb.Trans(y=50*1000,rot=180)),
        fhs_sub1_ref.ports['N1']
    ])
    
    pad_e9_moved = pad_ref.ports['e9'].copy()
    pad_e9_moved.center = (pad_e9_moved.center[0], fhs_ref.ports['W1'].center[1])
    gf.routing.route_single(
        c,
        fhs_sub2_ref.ports['N1'],
        pad_e9_moved,
        cross_section=xs_metal_wire,
        allow_width_mismatch=True,
        auto_taper=False,
        port_type='electrical',
        steps=[
            {'dx':0,'dy':-40},
            {'dx':-55,'dy':0},
            {'dx':0,'y':fhs_ref.ports['W1'].center[1]}
        ]
    )
    
    spring_pad = pad(size=(400, 400), metal_offset=10,pad_layer='PADDING')
    spring_pad_ref = c << spring_pad
    # spring_pad_ref.rotate(90)
    spring_pad_ref.movex(origin =spring_pad_ref.center[0] , destination=spring_ref.ports['up_anchor'].x)
    spring_pad_ref.movey(250)
    
    gf.routing.route_single(
        c,
        port1=spring_ref.ports['up_anchor'],
        port2=spring_pad_ref.ports['S1'],
        cross_section=xs_metal_wire,
        allow_width_mismatch=True,
        auto_taper=False,
        start_straight_length=15,
    )
     
    
    ring_beam_spacer = vertical_spacer(length=0.1)
    ring_ref = c << ring_resonator_fill_middle(gap=0.3, radius=10, length_x=3,length_y=0,cross_section=xs,bend="bend_circular", x_span=x_span_resonator)
    spacer_ref = c << ring_beam_spacer
    spacer_ref.connect('p1', beam_ref.ports['s1'],allow_width_mismatch=True)
    ring_ref.connect('p1', spacer_ref.ports['p2'],allow_width_mismatch=True)
    ring_ref.movex(beam_length/4)
    o1_ring = ring_ref['o1']
    o2_ring = ring_ref['o2']
    c.add_port(
        name='optical_in',
        width=o1_ring.width,
        orientation=-90,
        center=(o1_ring.center[0] - (waveguide_spacing- x_span_resonator)/2, o1_ring.center[1] -50),
        layer=o1_ring.layer,
        port_type='optical'
    )
    c.add_port(
        name='optical_out',
        width=o2_ring.width,
        orientation=-90,
        center=(o2_ring.center[0] + (waveguide_spacing- x_span_resonator)/2, o2_ring.center[1] -50),
        layer=o2_ring.layer,
        port_type='optical'
    )
    copied_portin = c.ports.filter(regex='optical_in')[0].copy()
    copied_portout = c.ports.filter(regex='optical_out')[0].copy()
    copied_portin.name = 'optical_in_reverse'
    copied_portout.name = 'optical_out_reverse'
    copied_portin.orientation = 90
    copied_portout.orientation = 90
    r1 = gf.routing.route_single(
        c,
        port1=o1_ring,
        port2=copied_portin,
        cross_section=xs_routing,
        )
    r2 = gf.routing.route_single(
        c,
        port1=o2_ring,
        port2=copied_portout,
        cross_section=xs_routing,
    )
    with do("create_deep_etch_mask(c,'bbox',mask_offset=mask_offset, deep_etch_layer='DEEP_ETCH_PL')").when(beam_fixed_support, "create_deep_etch_mask").goto("for port in rect.ports"):
        bfs_ref = c << beam_fixed_support(size=(200,200), metal_offset=3)
    bfs_ref.connect("E1", beam_ref.ports["e1"],allow_width_mismatch=True)
    
    c =merge_layers_with_priority(c, p)
    c = EBL_PL_overlap(c)
    c.info['params'] = {
        'beam_length': beam_length,
        'beam_width': beam_width,
        'spring_length': spring_length,
        'spring_width': spring_width,
        'finger_num': finger_num,
        'finger_num_sub': finger_num_sub,
    }
    
    return c 
c = beam_with_ring_resonator_3_combs(beam_length=500, beam_width=0.1,spring_length=15, spring_width=0.15,finger_num = 101, finger_num_sub=69)
c.show()

In [ ]:
device_list = [
    doubly_clamped_beam_with_ring_resonator(beam_length=500, beam_width=0.1,spring_length=15, spring_width=0.15,finger_num = 101),
    doubly_clamped_beam_with_ring_resonator(beam_length=500, beam_width=0.15,spring_length=15, spring_width=0.15,finger_num = 71),
    doubly_clamped_beam_with_ring_resonator(beam_length=500, beam_width=0.1,spring_length=15, spring_width=0.15,finger_num = 121),
    doubly_clamped_beam_with_ring_resonator(beam_length=500, beam_width=0.15,spring_length=15, spring_width=0.15,finger_num = 101),
    doubly_clamped_beam_with_ring_resonator(beam_length=500, beam_width=0.1,spring_length=15, spring_width=0.15,finger_num = 71),
    doubly_clamped_beam_with_ring_resonator(beam_length=500, beam_width=0.15,spring_length=15, spring_width=0.15,finger_num = 121),
    doubly_clamped_beam_with_ring_resonator(beam_length=500, beam_width=0.15,spring_length=15, spring_width=0.15,finger_num = 71),
]
device_list2 = [
    beam_with_ring_resonator_3_combs(beam_length=500, beam_width=0.1,spring_length=15, spring_width=0.15,finger_num = 101, finger_num_sub=71),
    beam_with_ring_resonator_3_combs(beam_length=500, beam_width=0.1,spring_length=15, spring_width=0.15,finger_num = 51, finger_num_sub=41),
    beam_with_ring_resonator_3_combs(beam_length=500, beam_width=0.1,spring_length=15, spring_width=0.15,finger_num = 101, finger_num_sub=51),
    beam_with_ring_resonator_3_combs(beam_length=500, beam_width=0.1,spring_length=15, spring_width=0.15,finger_num = 51, finger_num_sub=31),
    beam_with_ring_resonator_3_combs(beam_length=500, beam_width=0.1,spring_length=15, spring_width=0.15,finger_num = 71, finger_num_sub=31),
    beam_with_ring_resonator_3_combs(beam_length=500, beam_width=0.1,spring_length=15, spring_width=0.15,finger_num = 71, finger_num_sub=31),
    beam_with_ring_resonator_3_combs(beam_length=500, beam_width=0.1,spring_length=15, spring_width=0.15,finger_num = 71, finger_num_sub=31),
]

In [ ]:
import json
# @gf.cell
def full_device_array_with_grating_couplers(device_list:list):
    def dict_to_pretty_str(d):
        return json.dumps(d, indent=2)
    
    def device_array():
        text_spec = partial(text_outline, size=15, outline_width=0.5, with_mask=True,layer='MTOP', mask_layer='PROTECTION_PL',)
        c = gf.Component()
        device_refs = [c << d for d in device_list]
        for i, dev in enumerate(device_refs):
            dev.movex(i*1200-1297)
            labelme(c, dev, dict_to_pretty_str(device_list[i].info['params']), text_spec=text_spec, anchor="center", position=lambda d: (d.center[0]+400,d.ymin+600))
        c.ports = {}
        for i, dev in enumerate(device_refs):
            c.add_ports([dev.ports["optical_in"], dev.ports["optical_out"]])
        c.auto_rename_ports()
        return c
    c = gf.Component()
    my_coupler_spec = partial(my_coupler,coupler=gc_spec,waveguide_width=0.43)
    gc_array = gf.components.grating_coupler_array(my_coupler_spec, pitch=250, n=16, rotation=-90, with_loopback=True, cross_section=xs_routing).copy()
    create_deep_etch_mask(gc_array,'bbox',mask_offset=200,deep_etch_layer='PROTECTION_PL')
    gc_array_ref = c << gc_array
    gc_array_ref.move((-1000,-1000))
    device_array_ref = c << device_array()


    # align device array ports to grating coupler array ports
    device_array_ref.movex(
        origin=device_array_ref.ports['o8'].center[0],
        destination=gc_array_ref.ports[6].center[0]
    )
    device_array_ref_ports = [
        gf.Port(port.name, center=port.center, width=port.width, orientation=port.orientation, layer=port.layer)
        for port in device_array_ref.ports
    ]

    f = gf.routing.route_bundle(
        c,
        gc_array_ref.ports,
        device_array_ref_ports,
        cross_section=xs_routing,
        sort_ports=True,
        start_straight_length=200,
        separation=10
        
    )
    routing_bboxes = [inst.bbox() for route in f for inst in route.instances]
    for bbox in routing_bboxes:
        c.add_polygon(gf.kdb.Polygon(bbox).sized(50e3,70e3,2), layer='PROTECTION_PL')

    # c.flatten()
    return c
c = full_device_array_with_grating_couplers(device_list)
c.show()

In [ ]:
when(convert_to_printable, "hulls = reg.sized(-3e3,1)").do("hulls = reg.sized(-3e3,2)").goto("c_output = gf.Component()")

In [ ]:
c_printable = convert_to_printable(c)
c_printable.flatten()
c_printable.show()

In [ ]:
layout_final = gf.Component()
layout_final << die_with_alignment_marks(1.5e4,layers=['MARKER'])
for i in range(4):
    printable_layout = convert_to_printable(full_device_array_with_grating_couplers(device_list if i%2==0 else device_list2))
    c_printable_ref = layout_final << printable_layout
    c_printable_ref.move(c_printable_ref.center,(-2000,-5200+3400*i))
layout_final << frame(size=19000, layers=["DEEP_ETCH_EBL","DEEP_ETCH_PL","SHALLOW_ETCH","MTOP"])
layout_final.show()

In [ ]:
gc_rib = partial(my_coupler,gc_spec,deep_layer="SHALLOW_ETCH")
gc_normal = partial(my_coupler,gc_spec,deep_layer="DEEP_ETCH_EBL")
xs_routing = cross_section_with_sleeves(0.43,5,radius=20, radius_min=15,sleeve_layer="DEEP_ETCH_EBL")
euler_test_spec = partial(euler_test, grating_coupler_spec=gc_normal,cross_section=xs_routing)

def optical_test_array():
    c = gf.Component()
    text_spec = partial(gf.components.text_freetype, size=50, layer="MTOP", font="Consolas")
    ebta = c << euler_bend_test_array(euler_test_spec)
    ebta.move((0,0))
    labelme(c,ebta, text="bend_num:[5,10,15]", position=lambda c:(c.xmax+200, c.ymin+200), text_spec=text_spec,anchor='SW')
    cta = c << converter_test_array(partial(converter_test, grating_coupler_spec=gc_rib),pair_num_list=[10,20,50,60], deep_etch_layer="DEEP_ETCH_EBL")
    cta.move((0,-500))
    labelme(c,cta, text="pair_num:[10,20,50,60]", position="center", text_spec=text_spec,anchor='SW')
    st = spiral_test(grating_coupler_spec=gc_normal,cross_section=xs_routing)
    st_ref = c << st
    st_ref.move((0,-1100))
    labelme(c,st_ref, text=f"Spiral lengths:{st.info['Spiral lengths']}", position=lambda c:(c.xmax+200, c.ymin+500), text_spec=text_spec,anchor='SW')
    return c
(layout_final << optical_test_array()).move((3500,-4200))
(layout_final << optical_test_array()).move((3500,5800))
layout_final.show()

In [ ]:
(layout_final << cantilever_pullin_array()).move((3800,650))
(layout_final << cantilever_pullin_array()).move((3800,-2600))
layout_final.show()

In [ ]:
etch_depth_arrays = [layout_final << etch_depth_array(layers=['DEEP_ETCH_PL','DEEP_ETCH_EBL','SHALLOW_ETCH'],frame_layer='MTOP') for _ in range(4)]
for i, eda in enumerate(etch_depth_arrays):
    eda.move(
        eda.center,
        (-8000 if i % 2 == 0 else 8000, -2500 if i // 2 == 0 else 2500),
    )

alignment_array = [
    layout_final << litho_caliper_array(
        types=["EBL","PL","EBL","PL"], 
        layers=["DEEP_ETCH_EBL","DEEP_ETCH_PL","SHALLOW_ETCH","MTOP"], 
        frame_layer='MTOP') for _ in range(4)]
for i, aa in enumerate(alignment_array):
    aa.move(
        aa.center,
        (-8000 if i % 2 == 0 else 8000, -3500 if i // 2 == 0 else 3500),
    )

layout_final.show()